<a href="https://colab.research.google.com/github/Arkhemis/Coursera_Capstone/blob/main/Capstone_ThirdAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neighborhoods battle in Toronto
We first import the relevant libraries

In [314]:
!pip install geocoder
import requests
import re
import pandas as pd
import numpy as np
import geocoder 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

Then scraping the data from the Wikipedia page

In [315]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, "html.parser")
toronto_table = soup.find('table')

We extract the data needed and transform it into a dataframe

In [316]:
dico = {'Postal_Code': [],
        'Borough': [],
        'Neighborhood': []
}
for data in toronto_table.find_all('td'):
  if data.span.text == "Not assigned":
    pass
  else:
    #Appending Postal_Code
    dico['Postal_Code'].append(data.find('p').text[:3])
    #Appending Neighborhood
    dico['Neighborhood'].append(re.findall(r"\(([^\)]+)\)", data.text))
    #Appending Borough
    br = (data.span.text).split('(')[0]
    dico['Borough'].append(br)
df = pd.DataFrame.from_dict(dico)

#Just converting the list into string
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: ' '.join(x))
print(df.shape)
df.head()

(103, 3)


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


Using the geocoder package (but alas, not working)

In [317]:

#coords = None
#df["latitude"] = 0
#df["longitude"] = 0

#for i in range(len(df)):
#  while(coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(df["Postal_Code"][i]))
#    print(g)
#    coords = g.latlng
 
# df["latitude"][i] = coords[0]
#  df["longitude"][i] = coords[1]

Since the above is not working, let us rely on the online csv and merge it with our dataset

In [318]:
location = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
location = location.rename(columns = {"Postal Code": "Postal_Code"})
df = df.merge(location.set_index('Postal_Code'), on='Postal_Code')
df.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [319]:
#Let's check the boroughs
print(df["Borough"].unique())

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto',
                                             'EtobicokeNorthwest':'Etobicoke Northwest',
                                             'East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df["Borough"].unique(), df["Borough"].value_counts())

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East YorkEast Toronto'
 'Central Toronto' 'MississaugaCanada Post Gateway Processing Centre'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'EtobicokeNorthwest'
 'East TorontoBusiness reply mail Processing Centre969 Eastern']
['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East York/East Toronto'
 'Central Toronto' 'Mississauga' 'Etobicoke Northwest'] North York                24
Downtown Toronto          18
Scarborough               17
Etobicoke                 11
Central Toronto            9
West Toronto               6
East Toronto               5
York                       5
East York                  4
Queen's Park               1
East York/East Toronto     1
Etobicoke Northwest        1
Mississauga                1
Name: Borough, dtype: int64


In [320]:
import folium
address = "Toronto"
#First, let's find the coordinates of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latitude, longitude, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Pre-Processing

Here, we will be focusing on the Downtown Toronto borough.

In [321]:
CLIENT_ID = 'LIZWZBQQJFNW4XER0145UIZBUW5IBL4CRO3QOENC2025UR5G' 
CLIENT_SECRET = 'SPF1UKA5I1VIX5WBVEDGPHBGERY2NELCT1O1RA3TCKMI0R51' 
VERSION = '20180605' # Foursquare API Version
LIMIT = 100 

In [322]:
#Let's find the coordinates of the Toronto' Neighborhoods
df_downto = df[df['Borough'] == "North York"]
df_downto.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


Let's use this function to extract the informations regarding the venues in the different Neighborhoods

In [323]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's assign the venues to the neighborhoods

In [324]:
df_downto_venues = getNearbyVenues(df_downto.Neighborhood, df_downto.Latitude, df_downto.Longitude)
print("Shape:",df_downto_venues.shape)
df_downto_venues.head()

Parkwoods
Victoria Village
Lawrence Manor / Lawrence Heights
Don Mills
Glencairn
Don Mills Flemingdon Park
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Fairview / Henry Farm / Oriole
Northwood Park / York University
Bayview Village
Downsview CFB Toronto
York Mills / Silver Hills
Downsview
North Park / Maple Leaf Park / Upwood Park
Humber Summit
Willowdale / Newtonbrook
Downsview
Bedford Park / Lawrence Manor East
Humberlea / Emery
Willowdale
Downsview
York Mills West
Willowdale
Shape: (244, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Now, let's regroup the venues by Neighborhoods

In [325]:
df_downto_venues.groupby('Neighborhood').count()
#We can count the unique categories as well
print("There is", len(df_downto_venues['Venue Category'].unique()), "unique categories in our dataset")

There is 101 unique categories in our dataset


Now, let's turn our categories into dummy (binary) variables

In [326]:
df_downto_dummy = pd.get_dummies(df_downto_venues[["Venue Category"]], prefix="")
df_downto_dummy["Neighborhood"] = df_downto_venues["Neighborhood"]
#Moving the column to the first position
df_downto_dummy.insert(0, 'Neighborhood', df_downto_dummy.pop('Neighborhood'))
df_downto_dummy.head()
df_downto_venues["Neighborhood"].unique()

array(['Parkwoods', 'Victoria Village',
       'Lawrence Manor / Lawrence Heights', 'Don Mills', 'Glencairn',
       'Don Mills Flemingdon Park', 'Hillcrest Village',
       'Bathurst Manor / Wilson Heights / Downsview North',
       'Fairview / Henry Farm / Oriole',
       'Northwood Park / York University', 'Bayview Village',
       'Downsview CFB Toronto', 'Downsview',
       'North Park / Maple Leaf Park / Upwood Park', 'Humber Summit',
       'Willowdale / Newtonbrook', 'Bedford Park / Lawrence Manor East',
       'Humberlea / Emery', 'Willowdale', 'York Mills West'], dtype=object)

In [327]:
print("Before grouping by neighborhoods", df_downto_dummy.shape)
df_downto_grouped = df_downto_dummy.groupby('Neighborhood').mean().reset_index()
print("After grouping by neighborhoods", df_downto_grouped.shape)
df_downto_grouped.head()

Before grouping by neighborhoods (244, 102)
After grouping by neighborhoods (20, 102)


,Neighborhood,_Accessories Store,_Airport,_American Restaurant,_Art Gallery,_Arts & Crafts Store,_Asian Restaurant,_Athletics & Sports,_Bakery,_Bank,_Bar,_Baseball Field,_Basketball Court,_Beer Store,_Bike Shop,_Boutique,_Breakfast Spot,_Bridal Shop,_Bubble Tea Shop,_Burger Joint,_Burrito Place,_Bus Station,_Business Service,_Butcher,_Café,_Caribbean Restaurant,_Chinese Restaurant,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store,_Cosmetics Shop,_Cupcake Shop,_Deli / Bodega,_Department Store,_Dim Sum Restaurant,_Diner,_Discount Store,...,_Ice Cream Shop,_Indian Restaurant,_Intersection,_Italian Restaurant,_Japanese Restaurant,_Jewelry Store,_Juice Bar,_Liquor Store,_Locksmith,_Lounge,_Massage Studio,_Mediterranean Restaurant,_Middle Eastern Restaurant,_Miscellaneous Shop,_Mobile Phone Shop,_Movie Theater,_Park,_Pet Store,_Pharmacy,_Pizza Place,_Playground,_Plaza,_Pool,_Portuguese Restaurant,_Pub,_Ramen Restaurant,_Restaurant,_Salon / Barbershop,_Sandwich Place,_Shopping Mall,_Sporting Goods Shop,_Supermarket,_Sushi Restaurant,_Thai Restaurant,_Theater,_Toy / Game Store,_Trail,_Video Game Store,_Vietnamese Restaurant,_Women's Store
0,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.095238,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.047619,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00,0.0,0.00,0.095238,0.000000,0.0,0.0,0.0,0.000000,0.047619,0.0,0.00,0.047619,0.00,...,0.047619,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.047619,0.0,0.047619,0.0,0.047619,0.0,0.047619,0.047619,0.0,0.0,0.0,0.0,0.000000,0.0,0.047619,0.0,0.047619,0.047619,0.00,0.047619,0.047619,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.250000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.00,0.25,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,...,0.000000,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Bedford Park / Lawrence Manor East,0.0,0.0,0.037037,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.037037,0.000000,0.0,0.0,0.0,0.0,0.0,0.037037,0.037037,0.00,0.00,0.0,0.00,0.074074,0.037037,0.0,0.0,0.0,0.037037,0.000000,0.0,0.00,0.000000,0.00,...,0.000000,0.037037,0.0,0.074074,0.00,0.0,0.037037,0.037037,0.037037,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.037037,0.074074,0.0,0.0,0.0,0.0,0.037037,0.0,0.037037,0.0,0.074074,0.000000,0.00,0.000000,0.074074,0.037037,0.0,0.0,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.25,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,...,0.000000,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Don Mills Flemingdon Park,0.0,0.0,0.000000,0.05,0.0,0.05,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.05,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.050000,0.00,0.05,0.0,0.05,0.100000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.05,0.000000,0.05,...,0.000000,0.000000,0.0,0.050000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.100000,0.0,0.050000,0.000000,0.05,0.050000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


# Top Venues by Neighborhoods
Let's first define a function that will return the top venues

In [328]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [329]:
top_venues = 10

columns = ['Neighborhood']
#Let's create first the columns
for venue in range(top_venues):
  columns.append('{} Most Common Venue'.format(venue+1))
df_downto_sorted = pd.DataFrame(columns=columns)


df_downto_sorted['Neighborhood'] = df_downto_grouped['Neighborhood']

#Let's find the most common values now by Neighborhoods
for ind in np.arange(df_downto_grouped.shape[0]):
    df_downto_sorted.iloc[ind, 1:] = return_most_common_venues(df_downto_grouped.iloc[ind, :], top_venues)

df_downto_sorted.head()

,Neighborhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Bathurst Manor / Wilson Heights / Downsview North,_Coffee Shop,_Bank,_Deli / Bodega,_Park,_Pharmacy,_Pizza Place,_Mobile Phone Shop,_Bridal Shop,_Middle Eastern Restaurant,_Restaurant
1,Bayview Village,_Chinese Restaurant,_Café,_Bank,_Japanese Restaurant,_Dim Sum Restaurant,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
2,Bedford Park / Lawrence Manor East,_Italian Restaurant,_Sushi Restaurant,_Coffee Shop,_Sandwich Place,_Pizza Place,_Fast Food Restaurant,_Café,_Juice Bar,_Liquor Store,_Locksmith
3,Don Mills,_Caribbean Restaurant,_Café,_Gym,_Japanese Restaurant,_Department Store,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
4,Don Mills Flemingdon Park,_Gym,_Restaurant,_Coffee Shop,_Beer Store,_Bike Shop,_Chinese Restaurant,_Asian Restaurant,_Dim Sum Restaurant,_Supermarket,_Sporting Goods Shop


# Clustering
Now, we can cluster the neighborhoods, based on their features, i.e. their venues

In [330]:
from sklearn.cluster import KMeans
kclusters = 5

df_downto_cluster = df_downto_grouped.drop('Neighborhood', 1)

#Now, we train the model
kmeans = KMeans(n_clusters=kclusters, random_state=1962).fit(df_downto_cluster)

#Let's check the first labels
print(kmeans.labels_[0:10])
df_downto_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

[1 0 1 0 1 1 1 1 1 1]


Let's assign the clusters to the neighborhoods

In [331]:
df_downto_merged = df_downto

df_downto_merged = df_downto.join(df_downto_sorted.set_index('Neighborhood'), on='Neighborhood')
df_downto_merged["Cluster_Labels"] = pd.to_numeric(df_downto_merged["Cluster_Labels"], downcast='integer')
print(df_downto_merged.shape)
#Drop NAs in the case they are neighborhoods that do not have any venues
df_downto_merged = df_downto_merged[df_downto_merged["1 Most Common Venue"].notna()]

df_downto_merged.head() 

(24, 16)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,_Park,_Food & Drink Shop,_Fast Food Restaurant,_Women's Store,_Department Store,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,_French Restaurant,_Coffee Shop,_Hockey Arena,_Portuguese Restaurant,_Department Store,_Chocolate Shop,_Clothing Store,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1.0,_Clothing Store,_Furniture / Home Store,_Gift Shop,_Boutique,_Coffee Shop,_Event Space,_Vietnamese Restaurant,_Women's Store,_Accessories Store,_Cupcake Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,_Caribbean Restaurant,_Café,_Gym,_Japanese Restaurant,_Department Store,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,_Park,_Bakery,_Japanese Restaurant,_Italian Restaurant,_Women's Store,_Department Store,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping


# Vizualisation 

Now, we can visualise the newly created clusters on the map

In [332]:
import matplotlib.cm as cm
import matplotlib.colors as colors
address = "Downtown Toronto, Toronto"
#First, let's find the coordinates of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

#Let's set the clusters' colors
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_downto_merged['Latitude'], df_downto_merged['Longitude'], df_downto_merged['Neighborhood'], df_downto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster analysis
Neigbourhoods of the different clusters can be found below

In [342]:
from IPython.core import display as ICD
for i in range(kclusters):
  print("For cluster n°",i)
  ICD.display(df_downto_merged.loc[df_downto_merged['Cluster_Labels'] == i, df_downto_merged.columns[[1] + list(range(5, df_downto_merged.shape[1]))]])
  print("\n")

For cluster n° 0


,Borough,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
7,North York,0.0,_Caribbean Restaurant,_Café,_Gym,_Japanese Restaurant,_Department Store,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
39,North York,0.0,_Chinese Restaurant,_Café,_Bank,_Japanese Restaurant,_Dim Sum Restaurant,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store




For cluster n° 1


,Borough,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,North York,1.0,_Park,_Food & Drink Shop,_Fast Food Restaurant,_Women's Store,_Department Store,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping
1,North York,1.0,_French Restaurant,_Coffee Shop,_Hockey Arena,_Portuguese Restaurant,_Department Store,_Chocolate Shop,_Clothing Store,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
3,North York,1.0,_Clothing Store,_Furniture / Home Store,_Gift Shop,_Boutique,_Coffee Shop,_Event Space,_Vietnamese Restaurant,_Women's Store,_Accessories Store,_Cupcake Shop
10,North York,1.0,_Park,_Bakery,_Japanese Restaurant,_Italian Restaurant,_Women's Store,_Department Store,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping
13,North York,1.0,_Gym,_Restaurant,_Coffee Shop,_Beer Store,_Bike Shop,_Chinese Restaurant,_Asian Restaurant,_Dim Sum Restaurant,_Supermarket,_Sporting Goods Shop
27,North York,1.0,_Mediterranean Restaurant,_Golf Course,_Fast Food Restaurant,_Dog Run,_Pool,_Deli / Bodega,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant
28,North York,1.0,_Coffee Shop,_Bank,_Deli / Bodega,_Park,_Pharmacy,_Pizza Place,_Mobile Phone Shop,_Bridal Shop,_Middle Eastern Restaurant,_Restaurant
33,North York,1.0,_Clothing Store,_Coffee Shop,_Fast Food Restaurant,_Restaurant,_Food Court,_Juice Bar,_Jewelry Store,_Japanese Restaurant,_Bank,_Bakery
34,North York,1.0,_Furniture / Home Store,_Bar,_Coffee Shop,_Massage Studio,_Miscellaneous Shop,_Caribbean Restaurant,_Asian Restaurant,_American Restaurant,_Airport,_Comfort Food Restaurant
40,North York,1.0,_Airport,_Park,_Playground,_Women's Store,_Department Store,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping




For cluster n° 2


,Borough,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
52,North York,2.0,_Park,_Women's Store,_Caribbean Restaurant,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store,_Cosmetics Shop
66,North York,2.0,_Park,_Convenience Store,_Women's Store,_Caribbean Restaurant,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Cosmetics Shop




For cluster n° 3


,Borough,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
57,North York,3.0,_Furniture / Home Store,_Baseball Field,_Diner,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store,_Cosmetics Shop,_Cupcake Shop




For cluster n° 4


,Borough,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
50,North York,4.0,_Pizza Place,_Gym,_Women's Store,_Department Store,_Chocolate Shop,_Clothing Store,_Coffee Shop,_Comfort Food Restaurant,_Construction & Landscaping,_Convenience Store
